#### What are you trying to do in this notebook?
The first thing I did was to import some of the libraries that I would initially need to run the program, being numpy for algebraic expressions, pandas for dataframes, and matplotlib and seaborn for graphics. 
I then used the os library to go into the operating system and retrieve the three csv files that I would need to run the program.
I then used pandas to read the three csv files into the program and to convert them to dataframes, calling them train, test, and submission.
I used seaborn to analyse the target, being train[‘num_sold’]. When I analysed it, it could be seen that it was skewed and there were a number of outliers.
I then decided to remove the outliers in the hope that my predictions would improve.

#### What we learned while making this notebook?
Once I had converted the outliers to null values, I checked for null values. I dropped the rows that had null values and this had the effect of reducing the length of the train dataframe.
I created the variable, target, which would be used to make predictions.
I analysed the target once more and, once the outliers had been removed, the shape of the column of data had greatly improved.
It is at this point that I created a new dataframe, combi, which was the train dataframe with the target dropped, and the test dataframe appended to it.

#### Why are you trying it?
Finding out if a day fell on Easter was a bit trickier because I was required to find out when Easter fell on the train and test data. Once the holidays had been put in the appropriate columns, I used sklearn and created a for loop to ordinal encode all of the columns that were of dtype object.
I then used datetime to convert the date to a number in the newly created column, [‘date_num’] and then convert this number to an integer.
I then dropped the [‘date’] and [‘year’] columns because they don’t contribute any valuable information when making predictions.
Once the dataframe had been suitably prepared, I defined the X, y and X_test variables. The y variable is the target, the X variable is composed of combi up to the length of train, and the X_test variable is composed of combi from the length of train to the end.
I then used sklearn’s train_test_split to split the X and y variables up into training and validation sets.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os 
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print (os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2022/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2022/test.csv")
submission = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2022/sample_submission.csv")

In [ ]:
sns.distplot(train['num_sold'])

In [ ]:
for x in ['num_sold']:
    q75,q25 = np.percentile(train.loc[:,x],[75,25])
    intr_qr = q75-q25
 
    max = q75+(1.5*intr_qr)
    min = q25-(1.5*intr_qr)
 
    train.loc[train[x] < min,x] = np.nan
    train.loc[train[x] > max,x] = np.nan

In [ ]:
train['num_sold'].isnull().sum()

In [ ]:
train = train.dropna(axis = 0)
train

In [ ]:
target = train['num_sold']

sns.distplot(train['num_sold'])

In [ ]:
combi = train.drop(['num_sold'], axis=1).append(test)
combi

In [ ]:
combi = combi.drop(['row_id'], axis=1)
combi

In [ ]:
combi['date'] = pd.to_datetime(combi['date'], errors='coerce')
combi

In [ ]:
from datetime import datetime

combi["day_of_week"] = combi['date'].dt.dayofweek
combi["is_weekend"] = combi['day_of_week'] > 4

combi

In [ ]:
combi['is_weekend'] = combi['is_weekend']* 1
combi

In [ ]:
combi['year'] = combi['date'].dt.year
combi['month'] = combi['date'].dt.month
combi['day'] = combi['date'].dt.day

combi

In [ ]:
if combi['month'] is 12 and combi['day'] is 25:
    combi['xmas1'] = True
else:
    combi['xmas1'] = False

combi['xmas1'] = combi['xmas1'] * 1

combi

In [ ]:
if combi['month'] is 12 and combi['day'] is 26:
    combi['xmas2'] = True
else:
    combi['xmas2'] = False

combi['xmas2'] = combi['xmas2'] * 1

combi

In [ ]:
if combi['month'] is 1 and combi['day'] is 1:
    combi['new_year'] = True
else:
    combi['new_year'] = False

combi['new_year'] = combi['new_year'] * 1

combi

In [ ]:
if combi['year'] is 2015 and combi['month'] is 4 and combi['day'] is 5:
    combi['easter'] = True
elif combi['year'] is 2016 and combi['month'] is 3 and combi['day'] is 27:
    combi['easter'] = True
elif combi['year'] is 2017 and combi['month'] is 4 and combi['day'] is 16:
    combi['easter'] = True
elif combi['year'] is 2018 and combi['month'] is 4 and combi['day'] is 1:
    combi['easter'] = True
elif combi['year'] is 2019 and combi['month'] is 4 and combi['day'] is 21:
    combi['easter'] = True
else:
    combi['easter'] = False

combi['easter'] = combi['easter'] * 1

combi

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()

for col in combi:
    if combi[col].dtype=="object":
        combi[col] = enc.fit_transform(combi[col].values.reshape(-1,1))
combi

In [ ]:
import datetime 

combi['date_num'] = combi['date'].dt.strftime('%d%m%Y')
combi['date_num'] = combi['date_num'].astype(int)
combi

In [ ]:
combi.drop(['date'], axis=1, inplace=True)
combi.drop(['year'], axis=1, inplace=True)
combi

In [ ]:
y = target
X = combi[: len(train)]
X_test = combi[len(train) :]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
X_train.shape, X_val.shape, y_train.shape,y_val.shape, X_test.shape

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingRegressor

model = HistGradientBoostingRegressor().fit(X_train, y_train)
print(model.score(X_train, y_train))

In [ ]:
y_pred = model.predict(X_val)
print(model.score(X_val, y_val))

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_val, y_pred, squared=False)
rmse

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_val, y_pred, edgecolors=(0, 0, 0))
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:
preds = model.predict(X_test)
preds = preds.astype(int)
preds[preds < 0] = 0
preds

In [ ]:
submission.num_sold = preds
submission.to_csv('submission.csv', index=False)
submission = pd.read_csv("submission.csv")
submission

#### Did it work?
I checked the metrics.
I used matplotlib to graph the predicted values against the true values.
I then predicted on the test set.
Once I had predicted on the test set, I prepared the data for submission.

#### What did you not understand about this process?
Well, everything provides in the competition data page. I've no problem while working on it. If you guys don't understand the thing that I'll do in this notebook then please comment on this notebook.

#### What else do you think you can try as part of this approach?
Look at a notebook which presents feature engineering (based on the insights of this EDA) and a linear model which makes use of the features.